# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [23]:
!pip install Selenium
!apt-get install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (84.0.4147.105-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [24]:
wd = webdriver.Chrome("chromedriver", options= chrome_options)

## 필요 라이브러리 import

In [25]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image



## `scroll_down()`: 스크롤을 내리는 함수

In [116]:
def scroll_down():
  scroll_count = 0
  print("[scroll_down(): 스크롤 다운 시작]")

  last_height = wd.execute_script("return document.body.scrollHeight")
  after_click = False

  while True:
    print(f"[스크롤 다운: {scroll_count}]")
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    scroll_count +=1
    time.sleep(1)

    new_height = wd.execute_script("return document.body.scrollHeight")
    if last_height == new_height:
      if after_click is True:
        break
      else:
        try:
          more_button = wd.find_element_by_xpath('//*[@id="islmp"]/div/div/div/div/div[5]/input')
          if more_button.is_displayed():
            more_button.click()
            after_click = True
        except NoSuchElementException as e:
          print(e)
          break

    last_height = new_height



## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [129]:
def click_and_save(dir_name, index, img, img_list_length):
  global scraped_count
  try:
    img.click()
    wd.implicitly_wait(3)
    src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')
    if src.split('.')[-1] == 'png':
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".png")
      print(f" {index+1}/{img_list_length} PNG 이미지 저장")
    else:
      urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + ".jpg")
      print(f" {index+1}/{img_list_length} JPG 이미지 저장")
    
    scraped_count +=1
  except HTTPError as e:
    print(e)
    pass
  except ElementClickInterceptedException as e:
    print(e)
    pass


## `scraping()`: 구글 이미지 스크래핑 시작

In [115]:
def scraping(dir_name, query):
  global scraped_count

  url = f"https://www.google.com/search?q={query}&tbm=isch&hl=ko&hl=ko&safe=active&safe=active&tbs=isz%3Al&sa=X&ved=0CAEQpwVqFwoTCLDy9K3XwOsCFQAAAAAdAAAAABAC&biw=1148&bih=1163"
  

  wd.get(url)
  wd.maximize_window()

  scroll_down()

  div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
  img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
  print(img_list)

  for index, img in enumerate(img_list):
    try:
      click_and_save(dir_name, index, img, len(img_list))
    except ElementClickInterceptedException as e:
      print(e)
      wd.execute_script("window.scrollTo(0, window.scrollY + 100")
      time.sleep(1)
      click_and_save(dir_name, index, img, len(img_list))
    except ConnectionError as e:
      print(e)
      pass
    except URLError as e:
      print(e)
      pass
    except socket.timeout as e:
      print(e)
      pass
    except socket.gaierror as e:
      print(e)
      pass
    except ElementNotInteractableException as e:
      print(e)
      break
  try:
    print("[스크래핑 종료 ( 성공률 : %.2f%%)]" % (scraped_count / len(img_list) * 100.0))
  except ZeroDivisionError as e:
    print(e)
  
  wd.quit()




## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [103]:
def filter_and_remove(dir_name, query, filter_size):
  filtered_count = 0

  for index, file_name in enumerate(os.listdir(dir_name)):
    try:
      file_path = os.path.join(dir_name, file_name)
      img = Image.open(file_path)

      if img.width < filter_size and img.height < filter_size:
        img.close()
        os.remove(file_path)
        print(f"{index} 이미지 제거")
        filtered_count +=1
    except OSError as e:
      print(e)
      os.remove(file_path)
      filetered_count +=1

  print(f"[이미지 제거 갯수 : {filtered_count} / {scraped_count}]")

## 이미지 확인

In [144]:
socket.setdefaulttimeout(30)

scraped_count = 0

wd = webdriver.Chrome("chromedriver", options= chrome_options)

path='./'
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400)

검색어 입력: forest
[./forest 디렉토리 생성]
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[스크롤 다운: 11]
[<selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71e0494ade5a86dbeed49", element="23f20333-8c08-4245-8407-c89451041ae5")>, <selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71e0494ade5a86dbeed49", element="09da0f76-3e6c-41cd-a238-29b08476b9c9")>, <selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71e0494ade5a86dbeed49", element="af1eb21d-494a-4f1b-830a-fe690d54c2a9")>, <selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71e0494ade5a86dbeed49", element="bffe6be5-eafa-46e9-9a26-2feeee692b2b")>, <selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71e0494ade5a86dbeed49", element="46376c8e-428b-4292-8ca9-c20525da888a")>, <selenium.webdriver.remote.webelement.WebElement (session="3598f99d2fa71

In [141]:
!rm -rf forest


In [146]:
!ls forest/

102.jpg  178.jpg  262.jpg  358.jpg  450.jpg  550.jpg  639.jpg  705.jpg	787.jpg
103.jpg  179.jpg  263.jpg  360.jpg  451.jpg  560.jpg  63.jpg   706.jpg	789.jpg
108.jpg  17.jpg   264.jpg  361.jpg  455.jpg  565.jpg  641.jpg  707.jpg	790.jpg
109.jpg  181.jpg  268.jpg  363.jpg  45.jpg   566.jpg  642.jpg  713.jpg	791.jpg
114.jpg  184.jpg  269.jpg  365.jpg  460.jpg  567.jpg  643.jpg  71.jpg	795.jpg
118.jpg  186.jpg  26.jpg   366.jpg  470.jpg  568.jpg  645.jpg  722.jpg	797.jpg
119.jpg  190.jpg  277.jpg  367.jpg  471.jpg  575.jpg  646.jpg  724.jpg	798.jpg
121.jpg  192.jpg  278.jpg  369.jpg  478.jpg  580.jpg  649.jpg  726.jpg	801.jpg
122.jpg  193.jpg  27.jpg   370.jpg  479.jpg  582.jpg  64.jpg   728.jpg	805.jpg
125.jpg  194.jpg  280.jpg  372.jpg  487.jpg  586.jpg  653.jpg  729.jpg	806.jpg
126.jpg  195.jpg  281.jpg  378.jpg  489.jpg  589.jpg  658.jpg  733.jpg	809.jpg
12.jpg	 196.jpg  282.jpg  379.jpg  490.jpg  591.jpg  661.jpg  739.jpg	810.jpg
131.jpg  201.jpg  283.jpg  381.jpg  494.jpg  593.jpg  

In [147]:
from google.colab import files
files.download('./forest/102.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>